In [7]:
from lumibot.brokers import Alpaca
from lumibot.backtesting import YahooDataBacktesting
from lumibot.strategies import Strategy
from lumibot.traders import Trader
from datetime import datetime
from dotenv import load_dotenv
import os

In [4]:
#get the APIKEYS
load_dotenv()

API_KEY=os.getenv("Key")
API_SECRET=os.getenv("Secret")
BASE_URL=os.getenv("Endpoint")


In [6]:
AlpacaConfig = {
    "API_KEY": API_KEY,
    "API_SECRET": API_SECRET,
    "PAPER": True
}

In [8]:
broker=Alpaca(AlpacaConfig)

2024-07-14 01:03:32 | asyncio | INFO | [unknown] Waiting for the socket stream connection to be established, 
                method _stream_established must be called
2024-07-14 01:03:32 | alpaca.trading.stream | INFO | started trading stream
2024-07-14 01:03:32 | alpaca.trading.stream | INFO | starting trading websocket connection


2024-07-14 01:03:33 | alpaca.trading.stream | INFO | connected to: BaseURL.TRADING_STREAM_PAPER


In [9]:
class MLTrader(Strategy):
    def initialize(self, symbol="SPY", cash_at_risk=.5):
        self.symbol=symbol
        #the we determine how often we wanna make a trader by sleeptime
        self.sleeptime="24H"
        #then set where to start with the trader in every 24 hours
        self.last_trade = None
        self.cash_at_risk=cash_at_risk
    

    #position sizing and limiting
    def position_size(self):
        #making sure that we have good use of money when trading
        cash_left_in_account = self.get_cash()
        last_price = self.get_last_price(symbol="SPY")
        quantity = round(cash_left_in_account*self.cash_at_risk/last_price, 0)
        return cash_left_in_account, last_price, quantity
        

    def on_trading_iteration(self):
        #get the returns in position_size function
        cash_left_in_account, last_price, quantity = self.position_size()

        if cash_left_in_account > last_price:
            if self.last_trade == None:
                new_order = self.create_order(
                    self.symbol,
                    quantity,
                    "buy",
                    type="market"
                )
                self.submit_order(new_order)
                self.last_trade="buy"


In [12]:
#lets set up the trading instance with our strategy variable
strategy=MLTrader(
    name="mlstrat",
    broker=broker,
    parameters={
        "symbol": "SPY",
        "cash_at_risk": .5
    }
)

In [ ]:
# set up the backtesting here for our strategy
#step 1: get the start_date
backtesting_start_date=datetime(2023, 12, 15)
backtesting_end_date=datetime(2023, 12, 31)
"""
step 3: set up the backtesting with the data source
the data source here is YahooBackTesting class but if
you have your own data, you would typically use panda backtesting

The backtesting_params may include the budget and other stuff too
check here ```https://pypi.org/project/lumibot/1.3.2/```
"""
strategy.backtest(
    YahooDataBacktesting,
    backtesting_start_date,
    backtesting_end_date,
    parameters={
        "symbol": "SPY",
        "cash_at_risk": .5
    }
)
